In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score

data_folder = "data"
good_folder = os.path.join(data_folder, "good")
bad_folder = os.path.join(data_folder, "bad")
resize_dim = (128, 128)
batch_size = 32

def preprocess_image(image_path, resize_dim):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image_resized = cv2.resize(image, resize_dim, interpolation=cv2.INTER_AREA)
    image_equalized = cv2.equalizeHist(image_resized)
    image_blurred = cv2.GaussianBlur(image_equalized, (5, 5), 0)
    return image_blurred.astype('float32') / 255.0

def data_generator(folder_good, folder_bad, resize_dim, batch_size):
    good_files = [os.path.join(folder_good, f) for f in os.listdir(folder_good) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
    bad_files = [os.path.join(folder_bad, f) for f in os.listdir(folder_bad) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
    all_files = [(file, 1) for file in good_files] + [(file, 0) for file in bad_files]
    np.random.shuffle(all_files)
    
    while True:
        for i in range(0, len(all_files), batch_size):
            batch_files = all_files[i:i+batch_size]
            X_batch, y_batch = [], []
            for file_path, label in batch_files:
                image = preprocess_image(file_path, resize_dim)
                X_batch.append(image)
                y_batch.append(label)
            X_batch = np.expand_dims(np.array(X_batch), axis=-1)
            y_batch = tf.keras.utils.to_categorical(y_batch, num_classes=2)
            yield X_batch, y_batch

good_count = len([f for f in os.listdir(good_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))])
bad_count = len([f for f in os.listdir(bad_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))])
total_count = good_count + bad_count
steps_per_epoch = total_count // batch_size

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(resize_dim[0], resize_dim[1], 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

train_gen = data_generator(good_folder, bad_folder, resize_dim, batch_size)
model.fit(train_gen, steps_per_epoch=steps_per_epoch, epochs=10)

test_gen = data_generator(good_folder, bad_folder, resize_dim, batch_size)
X_test, y_test = next(test_gen)
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_true_labels, y_pred_labels)
print(f"Accuracy: {accuracy:.2f}")




c:\Users\yash0\anaconda3\envs\myenv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 46s 274ms/step - accuracy: 0.7904 - loss: 0.5411
Epoch 2/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 43s 268ms/step - accuracy: 0.8366 - loss: 0.3234
Epoch 3/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 43s 268ms/step - accuracy: 0.9103 - loss: 0.2214
Epoch 4/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 45s 282ms/step - accuracy: 0.9476 - loss: 0.1444
Epoch 5/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 47s 294ms/step - accuracy: 0.9600 - loss: 0.1177
Epoch 6/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 47s 294ms/step - accuracy: 0.9699 - loss: 0.0885
Epoch 7/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 44s 274ms/step - accuracy: 0.9807 - loss: 0.0635
Epoch 8/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 46s 285ms/step - accuracy: 0.9822 - loss: 0.0543
Epoch 9/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 41s 254ms/step - accuracy: 0.9859 - loss: 0.0389
Epoch 10/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 44s 274ms/step - accuracy: 0.9894 - loss: 0.0363
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step
Accuracy: 0.94


In [2]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Calculate metrics
accuracy = accuracy_score(y_true_labels, y_pred_labels)
precision = precision_score(y_true_labels, y_pred_labels, average='binary')
recall = recall_score(y_true_labels, y_pred_labels, average='binary')
f1 = f1_score(y_true_labels, y_pred_labels, average='binary')

# Print the metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


Accuracy: 0.94
Precision: 0.96
Recall: 0.96
F1 Score: 0.96
